In [1]:
import os, sys
currentdir = os.path.abspath('')
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

In [2]:
from main_package.utils import data_path_to_abs_path
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [4]:
df_all = pd.read_csv(
    data_path_to_abs_path("assistment/skill_builder_data_corrected_collapsed.csv"), 
    encoding='windows-1252', 
    dtype={
        "skill_name": str,
        "skill_id": str,
        "correct" : bool,
    },
)
df_all.head()

,Unnamed: 0,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,...,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,1,33022537,277618,64525,33139,51424,1,True,1,32454,...,0,3,32454,30799,NaN,26,0,NaN,1,1.0
1,2,33022709,277618,64525,33150,51435,1,True,1,4922,...,0,3,4922,30799,NaN,55,0,NaN,2,2.0
2,3,35450204,220674,70363,33159,51444,1,False,2,25390,...,0,3,42000,30799,NaN,88,0,NaN,1,1.0
3,4,35450295,220674,70363,33110,51395,1,True,1,4859,...,0,3,4859,30059,NaN,41,0,NaN,2,2.0
4,5,35450311,220674,70363,33196,51481,1,False,14,19813,...,3,4,124564,30060,NaN,65,0,0.0,3,3.0


In [8]:
skills = df_all["skill_id"][~pd.isnull(df_all["skill_id"])].astype(str).apply(lambda x: x.split("_"))

In [11]:
len(np.unique(np.concatenate(skills.reset_index(drop=True))))

123

In [9]:
df_all["count"] = df_all["skill_id"].astype(str).apply(lambda x: len(x.split("_")))
print(sum(df_all["count"]))

401756


In [10]:
df = df_all
print(df.columns)
df = df[["order_id", "user_id", "skill_id", "problem_id", "correct", "first_action"]]
df = df[~df["skill_id"].isna()]

#what is first_action in the data? -> probably 0 is for attempting
print(df.groupby("first_action").count())

# delete students with only one problem attempt
df_exclude = df.groupby("user_id").count()
print(df_exclude[df_exclude["order_id"] == 4])
df_exclude = df_exclude[df_exclude["order_id"] <= 1]
df = df[df["user_id"].apply(lambda uid: uid not in df_exclude.index)] #deletes around 30K rows

df = df[df["first_action"] == 0].drop(columns="first_action")
print(df)

Index(['Unnamed: 0', 'order_id', 'assignment_id', 'user_id', 'assistment_id',
       'problem_id', 'original', 'correct', 'attempt_count',
       'ms_first_response', 'tutor_mode', 'answer_type', 'sequence_id',
       'student_class_id', 'position', 'type', 'base_sequence_id', 'skill_id',
       'skill_name', 'teacher_id', 'school_id', 'hint_count', 'hint_total',
       'overlap_time', 'template_id', 'answer_id', 'answer_text',
       'first_action', 'bottom_hint', 'opportunity', 'opportunity_original',
       'count'],
      dtype='object')
              order_id  user_id  skill_id  problem_id  correct
first_action                                                  
0               255317   255317    255317      255317   255317
1                21565    21565     21565       21565    21565
2                 6223     6223      6223        6223     6223
         order_id  skill_id  problem_id  correct  first_action
user_id                                                       
77984      

In [11]:
df_test_users = df["user_id"].sample(frac=0.2)
print(df_test_users)
df_test = df[df["user_id"].apply(lambda uid: uid in df_test_users)].sort_values(by=["order_id"])
df_train = df[df["user_id"].apply(lambda uid: uid not in df_test_users)].sort_values(by=["order_id"])
#df_train.to_csv("train_data_v3.csv", index=False)
#df_test.to_csv("test_data_v3.csv", index=False)

82407     81946
208039    71918
164431    79617
121243    71826
116226    70708
          ...  
38596     96220
187915    81083
58331     96218
260987    96256
45008     79062
Name: user_id, Length: 51043, dtype: int64
